# Function Calling in OpenAI
https://platform.openai.com/docs/guides/function-calling/parallel-function-calling

In [2]:
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
prompt = "Hello, How are you?"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0125",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response["choices"][0]["message"]["content"])

Hello! I'm here and ready to assist you. How can I help you today?


# It contains all kind of API
https://rapidapi.com/hub

In [16]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      "x-rapidapi-key": "71527c9a2bmshce5ace3743dda8cp1c6596jsn726ecd8bb848",
      "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()


In [19]:
response = get_current_weather("Bangalore")

[{'name': 'Bengaluru', 'place_id': 'bengaluru', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '12.97194N', 'lon': '77.59369E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Bengaluru Urban', 'place_id': 'bangalore-urban', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '13.0N', 'lon': '77.58333E', 'timezone': 'Asia/Kolkata', 'type': 'administrative_area'}, {'name': 'Yelahanka', 'place_id': 'yelahanka', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '13.10073N', 'lon': '77.59632E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Anekal', 'place_id': 'anekal', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '12.7111N', 'lon': '77.69557E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Devanahalli', 'place_id': 'devanhalli', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Rural', 'country': 'India

In [20]:
response

[{'name': 'Bengaluru',
  'place_id': 'bengaluru',
  'adm_area1': 'Karnataka',
  'adm_area2': 'Bengaluru Urban',
  'country': 'India',
  'lat': '12.97194N',
  'lon': '77.59369E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Bengaluru Urban',
  'place_id': 'bangalore-urban',
  'adm_area1': 'Karnataka',
  'adm_area2': 'Bengaluru Urban',
  'country': 'India',
  'lat': '13.0N',
  'lon': '77.58333E',
  'timezone': 'Asia/Kolkata',
  'type': 'administrative_area'},
 {'name': 'Yelahanka',
  'place_id': 'yelahanka',
  'adm_area1': 'Karnataka',
  'adm_area2': 'Bengaluru Urban',
  'country': 'India',
  'lat': '13.10073N',
  'lon': '77.59632E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Anekal',
  'place_id': 'anekal',
  'adm_area1': 'Karnataka',
  'adm_area2': 'Bengaluru Urban',
  'country': 'India',
  'lat': '12.7111N',
  'lon': '77.69557E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Devanahalli',
  'place_id': 'devanhalli',
  'ad

In [8]:
# Connect the Api with LLm Models
# Json Format(Present in DOcumentation)
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [9]:

user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})

# Chat COmpletion Model
completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0125",
    messages= messages
    
)

In [10]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hello! How can I assist you today?",
  "refusal": null
}


In [11]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [36]:
# Calling the function

user_message="What is the temperature of Bangalore"

messages.append({"role": "user", "content": user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    functions=functions
    
)

In [39]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Bangalore\"}"
  },
  "refusal": null
}


In [40]:
response=completion.choices[0].message

In [41]:
response

<OpenAIObject at 0x184ccd951c0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Bangalore\"}"
  },
  "refusal": null
}

In [42]:
# Extract the function name
function_name=response['function_call']['name']
print(function_name)


get_current_weather


In [43]:
# Location
import json
location=eval(response['function_call']['arguments'])['location']
print(location)

Bangalore


In [44]:
# Step 4: send the info on the function call and function response to GPT
messages.append(response)  # extend conversation with assistant's reply
# Adding response to message as a prompt 
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [45]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Delhi'},
 <OpenAIObject at 0x184ccd85580> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"location\":\"Delhi\"}"
   },
   "refusal": null
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Delhi'},
 {'role': 'user', 'content': 'What is the temperature of Mumbai'},
 <OpenAIObject at 0x184ccd874c0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"location\":\"Mumbai\"}"
   },
   "refusal": null
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Mumbai'},
 {'role': 'user', 'content': 'What is the temperature of Bangalore'},
 <OpenAIObject at 0x184ccd951c0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"locati

In [46]:
# extend conversation with function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response


In [47]:
print(second_response.choices[0].message)

{
  "role": "assistant",
  "content": "Currently, the weather in Bangalore is clear with a temperature of 28\u00b0C.",
  "refusal": null
}
